In [1]:
!pip install --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from google.auth import default

creds, _ = default()
docs_service = build('docs', 'v1', credentials=creds)

In [5]:
MARKDOWN_TEXT = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [8]:
def create_google_doc_from_markdown(markdown_text: str):
    try:
        doc = docs_service.documents().create(
            body={"title": "Product Team Sync - May 15, 2023"}
        ).execute()
        document_id = doc["documentId"]

        requests = []
        cursor = 1

        def insert_text(text):
            nonlocal cursor
            requests.append({
                "insertText": {
                    "location": {"index": cursor},
                    "text": text
                }
            })
            cursor += len(text)

        def apply_style(start, end, style):
            requests.append({
                "updateParagraphStyle": {
                    "range": {"startIndex": start, "endIndex": end},
                    "paragraphStyle": style,
                    "fields": ",".join(style.keys())
                }
            })

        def apply_text_style(start, end, style):
            requests.append({
                "updateTextStyle": {
                    "range": {"startIndex": start, "endIndex": end},
                    "textStyle": style,
                    "fields": ",".join(style.keys())
                }
            })

        lines = markdown_text.strip().split("\n")

        for line in lines:
            start = cursor

            if line.startswith("# "):
                insert_text(line[2:] + "\n")
                apply_style(start, cursor, {"namedStyleType": "HEADING_1"})

            elif line.startswith("## "):
                insert_text(line[3:] + "\n")
                apply_style(start, cursor, {"namedStyleType": "HEADING_2"})

            elif line.startswith("### "):
                insert_text(line[4:] + "\n")
                apply_style(start, cursor, {"namedStyleType": "HEADING_3"})

            elif line.strip().startswith("- [ ]"):
                text = line.replace("- [ ]", "").strip() + "\n"
                insert_text(text)
                requests.append({
                    "createParagraphBullets": {
                        "range": {"startIndex": start, "endIndex": cursor},
                        "bulletPreset": "BULLET_CHECKBOX"
                    }
                })

                if "@" in text:
                    at_index = text.index("@")
                    apply_text_style(
                        start + at_index,
                        start + at_index + text[at_index:].split(":")[0].__len__(),
                        {
                            "bold": True,
                            "foregroundColor": {
                                "color": {
                                    "rgbColor": {"red": 0.2, "green": 0.4, "blue": 0.8}
                                }
                            }
                        }
                    )

            elif line.strip().startswith(("* ", "- ")):
                indent = line.count("  ")
                insert_text(line.strip()[2:] + "\n")
                requests.append({
                    "createParagraphBullets": {
                        "range": {"startIndex": start, "endIndex": cursor},
                        "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
                    }
                })
                if indent:
                    requests.append({
                        "updateParagraphStyle": {
                            "range": {"startIndex": start, "endIndex": cursor},
                            "paragraphStyle": {"indentStart": {"magnitude": indent * 18, "unit": "PT"}},
                            "fields": "indentStart"
                        }
                    })

            elif line.startswith(("Meeting recorded by", "Duration")):
                insert_text(line + "\n")
                apply_text_style(
                    start,
                    cursor,
                    {"italic": True, "fontSize": {"magnitude": 9, "unit": "PT"}}
                )

            else:
                insert_text(line + "\n")

        docs_service.documents().batchUpdate(
            documentId=document_id,
            body={"requests": requests}
        ).execute()

        return f"https://docs.google.com/document/d/{document_id}"

    except Exception as e:
        raise RuntimeError(f"Failed to create document: {e}")

In [9]:
doc_url = create_google_doc_from_markdown(MARKDOWN_TEXT)
doc_url

'https://docs.google.com/document/d/1tyjdb-98KcrYI_AEts7DguZxLdNt22198rFn5SX5dXY'